
\begin{aligned}
K_f = \frac{m_f \cdot a}{2 \cdot \theta_1} \\
K_r = \frac{m_r \cdot a}{2 \cdot \theta_2} 
\end{aligned}

\begin{aligned}
A=-\frac{m}{2 l^{2}} \frac{l_{f} K_{f}-l_{r} K_{r}}{K_{f} K_{r}}=-\frac{1}{l}\left(\frac{\theta_{2}}{a}-\frac{\theta_{1}}{a}\right)
\end{aligned}

\begin{aligned}
\omega_{n} & =\frac{2 l}{V} \sqrt{\frac{K_{f} K_{r}}{m I}} \sqrt{1+A V^{2}}
\end{aligned}



In [1]:
import json
from ipywidgets import interactive, Button, HBox, VBox, IntSlider
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---unique--------------------------------
omega_n_orig_values = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
A_orig_values = [1.0, 1.2, 1.4, 1.6 ,1.8, 2.0]

omega_n_lines = [x * 2 * np.pi for x in omega_n_orig_values]
A_lines = [x * 10**(-3) for x in A_orig_values]

# Function to create an interactive plot
def interactive_plot_func(mf, mr, l, I, V):
    l = l / 1000  # (mm -> m)
    V = V / 3.6  # (kph -> m/s)
    a = 4.0  # [m/s^2]
    m = mf + mr

    theta_1 = np.linspace(1.2/180*np.pi, 2.6/180*np.pi, 100)  # [deg/0.4G]
    theta_2 = np.linspace(0.2/180*np.pi, 1.6/180*np.pi, 100)  # [deg/0.4G]

    THETA_1, THETA_2 = np.meshgrid(theta_1, theta_2)

    K_f = (mf * a) / (2 * THETA_1)
    K_r = (mr * a) / (2 * THETA_2)

    A = -(1/l) * (THETA_2/a - THETA_1/a)
    omega_n = np.nan_to_num((2 * l / V) * np.sqrt((K_f * K_r) / (m * I)) * np.sqrt(1 + A * V**2), nan=0.0, posinf=0.0, neginf=0.0)


    fig, ax = plt.subplots()

    # Functions to customize label formats
    def format_omega_n_label(x):
        return f"ωn={x / 2 / np.pi:.1f}"

    def format_A_label(x):
        return f"SF={x / 10**(-3):.1f}"

    dataframes = []
    for omega_n_line in omega_n_lines:
        CS = ax.contour(THETA_2 * 180/np.pi, THETA_1 * 180/np.pi, omega_n, levels=[omega_n_line], colors='blue', linestyles='solid')
        ax.clabel(CS, inline=1, fontsize=10, fmt=format_omega_n_label)
        for seg in CS.allsegs[0]:
            df = pd.DataFrame(seg, columns=['theta_2', 'theta_1'])
            df['omega_n'] = omega_n_line
            dataframes.append(df)

    for A_line in A_lines:
        CS = ax.contour(THETA_2 * 180/np.pi, THETA_1 * 180/np.pi, A, levels=[A_line], colors='red', linestyles='dashed')
        ax.clabel(CS, inline=1, fontsize=10, fmt=format_A_label)
        for seg in CS.allsegs[0]:
            df = pd.DataFrame(seg, columns=['theta_2', 'theta_1'])
            df['A'] = A_line
            dataframes.append(df)

    ax.set_xlabel('Rr slip angle theta_2 [deg/0.4G]')
    ax.set_ylabel('Fr slip angle theta_1 [deg/0.4G]')
    ax.set_title('omega_n and A lines')
    ax.grid()

    plt.show()
    df_all = pd.concat(dataframes)
    return df_all

# Function to define the operation of the Save data button
def save_data(b):
    df_all = interactive_plot_func(**{child.description.split(' ')[0]: child.value for child in interactive_plot.children[:-1]})

    # Create a new dataframe to store theta_1 and theta_2 values for each omega_n and A
    df_to_save = pd.DataFrame()

    for idx, omega_n_val in enumerate(omega_n_lines):
        df_omega = df_all[df_all["omega_n"] == omega_n_val]
        df_omega = df_omega.rename(columns={"theta_1": f"theta_1_omega={omega_n_orig_values[idx]}", "theta_2": f"theta_2_omega={omega_n_orig_values[idx]}"})
        df_omega = df_omega.drop(columns=["omega_n", "A"])
        df_to_save = pd.concat([df_to_save, df_omega], axis=1)

    for idx, A_val in enumerate(A_lines):
        df_A = df_all[df_all["A"] == A_val]
        df_A = df_A.rename(columns={"theta_1": f"theta_1_A={A_orig_values[idx]}", "theta_2": f"theta_2_A={A_orig_values[idx]}"})
        df_A = df_A.drop(columns=["omega_n", "A"])
        df_to_save = pd.concat([df_to_save, df_A], axis=1)

    df_to_save.to_csv('data.csv', index=False)

    
# ---partially unique--------------------------------

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "mf": {"name": "Front mass", "unit": "kg"},
    "mr": {"name": "Rear mass", "unit": "kg"},
    "l": {"name": "Wheelbase", "unit": "mm"},
    "I": {"name": "Yaw inertia", "unit": "kgm^2"},
    "V": {"name": "Velocity", "unit": "kph"},
}

# Creating the sliders
spec_sliders = {
    "mf": IntSlider(min=500, max=2000, step=10, value=1000, description=f'mf [{slider_parameters["mf"]["unit"]}]', continuous_update=False),
    "mr": IntSlider(min=500, max=2000, step=10, value=700, description=f'mr [{slider_parameters["mr"]["unit"]}]', continuous_update=False),
    "l": IntSlider(min=2000, max=4000, step=10, value=2700, description=f'l [{slider_parameters["l"]["unit"]}]', continuous_update=False),
    "I": IntSlider(min=2000, max=4000, step=10, value=2650, description=f'I [{slider_parameters["I"]["unit"]}]', continuous_update=False)
}
condition_sliders = {
    "V": IntSlider(min=30, max=120, step=30, value=120, description=f'V [{slider_parameters["V"]["unit"]}]', continuous_update=False)
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))